In [2]:
import os
import keras
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers.merge import concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import  Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D, LSTM
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def load_cnn_model(model_path='./result/test'):
    with open(os.path.join(model_path, 'model_architecture.json'), 'r') as f:
        model = model_from_json(f.read())

    model.load_weights(os.path.join(model_path, 'weights-42-0.90.hdf5'))
    
    return model

In [4]:
def load_rnn_model(model_path='./result/rnn_model'):
    with open(os.path.join(model_path, 'model_architecture.json'), 'r') as f:
        model = model_from_json(f.read())

    model.load_weights(os.path.join(model_path, 'weights-improvement-97-0.86.hdf5'))
    
    return model

In [5]:
def load_data(dataset_path='./npy_files'):
    X = np.load(os.path.join(dataset_path,'X.npy'))
    y = np.load(os.path.join(dataset_path,'y.npy'))
    folds = np.load(os.path.join(dataset_path,'folds.npy'))
    feature_class = np.load(os.path.join(dataset_path, 'feature_class.npy'))
    facs = np.load(os.path.join(dataset_path,'facs_time.npy'))

    # rescale [0,255] -> [0,2]    
    X = X.astype('float32')/255*2

    # one-hot encode the labels
    num_classes = len(np.unique(y))
    y = keras.utils.to_categorical(y, num_classes)

    return X, y, folds, feature_class, facs

In [6]:
def make_facs_y(facs):
    facs_y = []
    for inst in facs:
        temp = np.zeros((15, 65))
        temp[:-1] = inst[1:]
        temp[-1][0] = 1
        facs_y.append(temp)
    facs_y = np.array(facs_y)
    
    return facs_y

In [7]:
def make_cv_folds(X, y, feature_class, facs, folds, fold_num):
    X_train = []
    y_train = []
    feature_class_train = []
    facs_train = []
    
    X_valid = []
    y_valid = []
    feature_class_valid = []
    facs_valid = []
    
    for index, fold in enumerate(folds):
        if fold == fold_num:
            X_valid.append(X[index])
            y_valid.append(y[index])
            feature_class_valid.append(feature_class[index])
            facs_valid.append(facs[index])
        else:
            X_train.append(X[index])
            y_train.append(y[index])
            feature_class_train.append(feature_class[index])
            facs_train.append(facs[index])
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_valid = np.array(X_valid)
    y_valid = np.array(y_valid)
    
    feature_class_train = np.array(feature_class_train)
    feature_class_valid = np.array(feature_class_valid)

    facs_train = np.array(facs_train)
    facs_valid = np.array(facs_valid)
    facs_y_train = make_facs_y(facs_train)
    facs_y_valid = make_facs_y(facs_valid)

    return (X_train, y_train, 
            X_valid, y_valid,
            feature_class_train, feature_class_valid,
            facs_train, facs_valid,
            facs_y_train, facs_y_valid
           )

In [8]:
cnn_model = load_cnn_model()
rnn_model = load_rnn_model()

OSError: Unable to open file (unable to open file: name = './result/rnn_model/weights-improvement-97-0.86.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [24]:
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 64, 96, 96)        1600      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 48, 96)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 48, 96)       102400    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 64, 24, 96)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 256, 24, 96)       409600    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 21, 2, 96)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4032)              0         
__________

In [9]:
X, y, folds, feature_class, facs = load_data('./save_data/npy_files/')

In [10]:
X_train, y_train, X_valid, y_valid, feature_class_train, feature_class_valid,facs_train, facs_valid,facs_y_train, facs_y_valid = make_cv_folds(X, y, feature_class,
                                            facs, folds, 0)

In [19]:
facs.shape

(1307, 15, 65)

In [20]:
X

array([[[[0.08627451, 0.07843138, 0.08627451, ..., 0.        ,
          0.02352941, 0.8       ],
         [0.03921569, 0.03921569, 0.03137255, ..., 0.        ,
          0.        , 0.42352942],
         [0.04705882, 0.04705882, 0.03921569, ..., 0.        ,
          0.        , 0.02352941],
         ...,
         [0.7529412 , 0.7294118 , 0.6745098 , ..., 1.1686275 ,
          1.2078432 , 1.2705883 ],
         [0.6431373 , 0.63529414, 0.57254905, ..., 1.1686275 ,
          1.0196079 , 1.0196079 ],
         [0.6509804 , 0.49411765, 0.4627451 , ..., 1.1529412 ,
          1.2313726 , 1.2941177 ]]],


       [[[0.04705882, 0.07058824, 0.07843138, ..., 0.03921569,
          0.07843138, 0.05490196],
         [0.04705882, 0.03921569, 0.03921569, ..., 0.03921569,
          0.07058824, 0.03137255],
         [0.03137255, 0.03137255, 0.02352941, ..., 0.02352941,
          0.05490196, 0.03921569],
         ...,
         [0.7137255 , 0.6745098 , 0.627451  , ..., 1.1058824 ,
          1.0980393 , 1

In [33]:
facs.shape

(1307, 15, 65)

In [27]:
rnn_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 15, 65)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 15, 250)      316000      input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 15, 4040)     0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 15, 4290)     0           lstm_1[0][0]                     
                                                                 input_3[0][0]                    
__________

In [23]:
feature_class.shape

(1307, 15, 4040)

In [35]:
facs_prob = rnn_model.predict([facs_valid, feature_class_valid])

In [109]:
facs_classes = facs_prob.argmax(axis=2)
facs_valid = facs_y_valid.argmax(axis=2)

In [42]:
facs_classes.shape

(132, 15)

In [104]:
facs_classes[2]

array([ 9, 17,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [110]:
facs_predict = facs_classes.reshape(-1,)
facs_valid = facs_valid.reshape(-1,)

In [111]:
facs_predict.shape

(1980,)

In [112]:
facs_valid.shape

(1980,)

In [21]:
count = 0
count_true = 0
for index, valid in enumerate(facs_valid):
    if valid != 0:
        count += 1
    print(facs_predict[index], valid)
    if facs_predict[index] == valid and valid != 0:
        count_true += 1

print(count_true)
print(count_true/count)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()